In [13]:
import sqlite3 as sql
import numpy as np
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

NUM_WORDS = 10000
SEED = 113

data = sql.connect("movie_lines.db")
cursor = data.cursor()

## Gather data

In [14]:
cursor.execute("SELECT COUNT(movie_id) FROM movies")
count = cursor.fetchone()[0]

movie_lines = []
for i in range(count):
    cursor.execute("SELECT line_text FROM lines WHERE movie_id = {}".format(i))
    lines = ""
    lines = " ".join([line[0] for line in cursor.fetchall()])
    movie_lines.append(lines)
    

In [19]:
tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token="<UNK>")
tokenizer.fit_on_texts(movie_lines)

In [20]:
data = np.array(tokenizer.texts_to_sequences(movie_lines))

## Gather labels

In [15]:
cursor.execute("SELECT name FROM genres")
all_genres = np.array([genre[0] for genre in cursor.fetchall()])

label_encoder = LabelEncoder()
genre_ints = label_encoder.fit_transform(all_genres)

genre_dict = {}
for i in range(len(all_genres)):
    genre_dict[all_genres[i]] = genre_ints[i]

In [ ]:
genre_dict

In [16]:
movie_genres = []
for i in range(count):
    cursor.execute("""
        SELECT genres.name
        FROM movies, genres, movie_genre_linking
        WHERE movies.movie_id = movie_genre_linking.movie_id
          AND genres.genre_id = movie_genre_linking.genre_id
          AND movies.movie_id = {}
        """.format(i))
    genres = [genre[0] for genre in cursor.fetchall()]
    movie_genres.append(genres)

In [ ]:
movie_genres

In [17]:
movie_genres_int = []
for entry in movie_genres:
    int_encoded = []
    for genre in entry:
        int_encoded.append(genre_dict[genre])
    movie_genres_int.append(int_encoded)

In [18]:
labels = np.array(movie_genres_int)

## Randomize and Split

In [23]:
# Based on keras.datasets.imdb implementation of shuffling
np.random.seed(SEED)
indices = np.arange(len(data))
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

In [24]:
train_data = data[:462]
train_labels = labels[:462]
test_data = data[462:]
test_labels = labels[462:]

## One-Hot Encode

In [ ]:
# From https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/3.5-classifying-movie-reviews.ipynb
def vectorize_sequences(sequences, dimension=NUM_WORDS):
    # Create an all-zero matrix of shape (len(sequences), dimension)
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # set specific indices of results[i] to 1s
    return results

x_train = vectorize_sequences(features)